In [2]:
import pandas as pd
use_cols = ["User_id","Movie_id","Ratings","Timestamp"]
df = pd.read_csv('/Users/himangini/Desktop/CMPE-257/ml-1m/ratings.dat', sep='::', names=use_cols, engine='python')
df.head(5)

,User_id,Movie_id,Ratings,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
df2 = df.sample(n=20000)
df2.shape

(20000, 4)

In [4]:
df2.info

<bound method DataFrame.info of         User_id  Movie_id  Ratings   Timestamp
609858     3696       431        3   966303314
638690     3841      2949        5   965995322
51119       337      3254        2   976348492
779847     4658         9        1   963967706
855733     5138      2644        3   962062007
33142       224       300        4   976833937
729471     4362      2410        2   965187800
722198     4323      2890        5   965250667
976116     5886      2058        3   957485795
963263     5809      2109        4   957987343
701424     4204       318        5   965321530
760422     4517      2746        4   974438652
381741     2231      3534        4   974607316
445060     2745      2091        3   973215970
175780     1112      1035        5   974942957
59074       402      2262        4   976296134
158970     1016      2708        4   975009476
339532     2000       364        4   974679762
244646     1473      1876        3   974757564
972822     5862       474   

In [5]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df2, test_size=0.2)

In [6]:
# Create two user-item matrices, one for training and another for testing
train_data_matrix = train_data.as_matrix(columns = ['User_id', 'Movie_id', 'Ratings'])
test_data_matrix = test_data.as_matrix(columns = ['User_id', 'Movie_id', 'Ratings'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0

In [8]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0

In [9]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [10]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

In [13]:
from math import sqrt
from sklearn.metrics import mean_squared_error
# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [14]:
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))

User-based CF RMSE: 967.7100236374531
Item-based CF RMSE: 106.22116150640953
